In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE TemplateHaskell #-}

import Network.HTTP.Req
import qualified Data.ByteString as B
import qualified Data.ByteString.Lazy as BL
import Control.Lens hiding ((.=))
import Data.Default.Class
import Data.Aeson
import Data.Aeson.Encode.Pretty
import Data.Aeson.Lens
import Data.Aeson.Text
import Data.Attoparsec.Text
import Data.Monoid ((<>), mconcat)
import qualified Data.Text as T
import qualified Data.Text.IO as T
import Data.Traversable
import Data.List (nub)
import GHC.Generics
import System.Process

In [2]:
data Project = Project
    { owner  :: T.Text
    , repo   :: T.Text
    , rev    :: T.Text
    , sha256 :: T.Text
    } deriving (Show, Generic)

instance FromJSON Project
instance ToJSON Project

In [3]:
Just versions <- decode <$> BL.readFile "versions.json" :: IO (Maybe Value)

In [4]:
extractProject :: Value -> T.Text -> Maybe Project
extractProject versions name = do
    prj <- versions ^? key name
    case fromJSON prj of
        Error _ -> Nothing
        Success p -> Just p

prj = extractProject versions "nixpkgs"
prj
toJSON prj

Just (Project {owner = "NixOS", repo = "nixpkgs-channels", rev = "cd0cd946f37cbf94f4a7893a8ce3da61c4dc3de0", sha256 = "0lcfch1bmgx907444qdk0bka1wnsc2bgyrlgm93g2s4djl8j07s8"})

Object (fromList [("owner",String "NixOS"),("repo",String "nixpkgs-channels"),("sha256",String "0lcfch1bmgx907444qdk0bka1wnsc2bgyrlgm93g2s4djl8j07s8"),("rev",String "cd0cd946f37cbf94f4a7893a8ce3da61c4dc3de0")])

In [5]:
versions & key "nixpkgs" .~ toJSON prj

Object (fromList [("nixpkgs",Object (fromList [("owner",String "NixOS"),("repo",String "nixpkgs-channels"),("sha256",String "0lcfch1bmgx907444qdk0bka1wnsc2bgyrlgm93g2s4djl8j07s8"),("rev",String "cd0cd946f37cbf94f4a7893a8ce3da61c4dc3de0")])),("ihaskell",Object (fromList [("owner",String "gibiansky"),("repo",String "IHaskell"),("sha256",String "041v7zlgvwf5bbvi52n2cp4kld0x7lcdij2fss7076bxkc8k0ayy"),("rev",String "0d454acc4d7752ce16bf7f321f308f602f6555ea")]))])

In [6]:
r :: (MonadHttp m, FromJSON a) => Project -> T.Text -> m (JsonResponse a)
r project branch = req GET
    (  https "api.github.com"
    /: "repos"
    /: owner project
    /: repo project
    /: "branches"
    /: branch
    ) NoReqBody jsonResponse (header "User-Agent" "vaibhavsagar")

In [7]:
Just project = prj
res <- responseBody <$> runReq def (r project "nixos-18.03") :: IO Value

In [8]:
res ^? key "commit" . key "sha" . _String

Just "cd0cd946f37cbf94f4a7893a8ce3da61c4dc3de0"

In [9]:
getRev :: Project -> T.Text -> IO (Maybe T.Text)
getRev project branch = do
    res <- responseBody <$> runReq def (r project branch) :: IO Value
    return $ res ^? key "commit" . key "sha" . _String
getRev project "nixos-18.03"

Just "cd0cd946f37cbf94f4a7893a8ce3da61c4dc3de0"

In [10]:
buildURL :: Project -> T.Text
buildURL project = "https://github.com/" <> owner project <> "/" <> repo project <> "/" <> "archive" <>"/" <> rev project <> ".tar.gz" 

getSha256 :: T.Text -> Bool -> IO String
getSha256 url unpack = let
    option = if unpack then ["--unpack"] else []
    in init <$> readProcess "nix-prefetch-url" (option ++ [T.unpack url]) ""

getSha256 (buildURL project {rev = "c1ef96ebdbd12fa5c9f0c6d241dd899137481350"}) False

"1clfwc0yjcc3qb3zhakkcp0w17fxjzcf63f8bb5q8yghfqq7rajy"

In [11]:
update :: String -> String -> String -> Bool -> IO ()
update filename projectName branchName unpack = do
    Just versions <- decode <$> BL.readFile filename :: IO (Maybe Value)
    let (Just project) = extractProject versions (T.pack projectName)
    Just latestRev <- getRev project (T.pack branchName)
    latestSha256 <- T.pack <$> getSha256 (buildURL project { rev = latestRev }) unpack
    let project' = project { rev = latestRev, sha256 = latestSha256 }
    let versions' = versions & key (T.pack projectName) .~ toJSON project'
    BL.writeFile filename (encodePretty versions')

In [12]:
update "versions.json" "nixpkgs" "nixos-18.03" False